In [8]:
import sys
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/")
sys.path.append("/Users/skrishna/Documents/phd_codes/neurips_paper/")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/Generative_Model")


sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR/recourse/")



In [31]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch.nn as nn
import pickle as pkl
from numpy import linalg as LA
import torch.nn as nn
import seaborn
import ML_Models.data_loader as loader
# from utils import _get_input_subset
# from Recourse_Methods.gradient_methods import SCFE
# from utils import get_recourses, get_performance_measures


%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [10]:
import numpy as np
from sklearn.manifold import TSNE

In [20]:
## Dataset Prep


from torchvision import  datasets, transforms
from torch.utils.data import DataLoader

# data_name = "compas"
data_name = "adult"
# data_name = "german"
n_starting_instances = 5000
compas_dict = {
        "data_path": '../Data_Sets/COMPAS/',
        "filename_train": 'compas-train-upsampled.csv',
        "filename_test": 'compas-test-upsampled.csv',
        "label": "risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }

german_dict = {
        "data_path": '../Data_Sets/German_Credit_Data/',
        "filename_train": 'german-train.csv',
        "filename_test": 'german-test.csv',
        "label": "credit-risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    
    }

adult_dict = {
        "data_path": "../Data_Sets/Adult/",
        "filename_train": 'adult-train.csv',
        "filename_test": 'adult-test.csv',
        "label": 'income',
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 25,
        "H2": 25,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }


data_meta_dictionaries = {
        "compas": compas_dict, 
        "adult": adult_dict, 
        "german":german_dict
    }
data_meta_info = data_meta_dictionaries[data_name]


dataset_test = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                 filename=data_meta_info["filename_test"],
                                                 label=data_meta_info["label"], scale = "standard")
        
dataset_train = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                  filename=data_meta_info["filename_train"],
                                                  label=data_meta_info["label"], scale = "standard")


column_names = pd.read_csv(data_meta_info["data_path"] + data_meta_info["filename_train"]).drop(data_meta_info["label"], axis=1).columns



In [21]:


# Data loader

train_loader = DataLoader(dataset_train, batch_size = 1, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size = 1, shuffle=False)

data_X = [i for i in train_loader]
# num_input = len(data[0][0][0])



epsilons = [0, 0.05, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.75, 1.0]



In [40]:
X = np.array([i[0].squeeze().numpy().tolist() for i in data_X])
X_embed = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(X[:1000])
for eps_ind,epsilon in enumerate(epsilons):
    model_robust = torch.load("./models/{}_lr_model_robust_eps_{}.pth".format(data_name, epsilon))
    Y = [int(model_robust(i[0].float()).item()>0) for i in data_X]
    X_val = [i[0] for i in X_embed]
    Y_val = [i[1] for i in X_embed]
    hue_val = [Y[i] for i in range(len(Y_val))] 
    pd_vis = pd.DataFrame(zip(X_val, Y_val, hue_val), columns = ["x", "y", "prediction"])
    seaborn.scatterplot(data=pd_vis, x="x", y="y", hue="prediction", style="prediction")
    plt.savefig("plots/{}_tsne_{}_{}.png".format(eps_ind, data_name, epsilon))
    plt.close()
#     plt.plot(X, Y)
    

In [ ]:
data = [i[0].numpy() for i in data]

In [17]:
data

[array([-0.87504854,  1.35344178, -0.43642485, -0.14663622, -0.21846736,
        -0.24818369,  0.69657768, -1.67502288,  0.93663052,  0.91052794,
         0.83816894,  0.39981721,  0.30877718]),
 array([-0.34534696, -0.1641928 , -0.82673761, -0.14663622, -0.21846736,
        -0.08181582,  0.69657768, -1.67502288, -1.06765686,  0.91052794,
        -1.19307689,  0.39981721,  0.30877718]),
 array([-0.19400366,  0.16894557,  1.12482621,  1.89353719, -0.21846736,
         1.99778257,  0.69657768,  0.59700677, -1.06765686,  0.91052794,
        -1.19307689,  0.39981721,  0.30877718]),
 array([ 0.63838454, -0.7417736 , -0.04611208, -0.14663622, -0.21846736,
        -0.49773549, -1.43559007,  0.59700677,  0.93663052,  0.91052794,
         0.83816894, -2.50114294, -3.23858133]),
 array([ 0.7140562 , -1.05719175,  1.51513898, -0.14663622, -0.21846736,
         1.58186289,  0.69657768, -1.67502288,  0.93663052, -1.09826394,
         0.83816894,  0.39981721,  0.30877718]),
 array([ 2.07614597, -0.7

In [7]:


X_embedded = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(X)

NameError: name 'X' is not defined